In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pandas as pd
import numpy as np
import re

In [2]:
# Load data
df = pd.read_csv('./data/product_df.csv')
df = df[['Star Rating', 'Comment']]

In [3]:
def label_sentiment(rating):
    if rating in [1, 2]:
        return 'tiêu cực'
    elif rating == 3:
        return 'trung tính'
    elif rating in [4, 5]:
        return 'tích cực'
    else:
        return 'không rõ'  # Nếu có xếp hạng nằm ngoài khoảng 1-5

# Gắn nhãn cảm xúc cho mỗi đánh giá
df['Sentiment'] = df['Star Rating'].apply(label_sentiment)

# Hiển thị 5 hàng đầu tiên của dataframe với cột sentiment mới
print(df.head())

   Star Rating                                            Comment   Sentiment
0            4                      Điện thoại này dùng rất thích    tích cực
1            4                               sử dụng thấy cũng ok    tích cực
2            2                      Bảo hành ít quá, chỉ 12 tháng    tiêu cực
3            5                             Sản phẩm mượt, chạy êm    tích cực
4            3  Cho mình hỏi muốn khởi động lại máy hay tắt ng...  trung tính


In [4]:
# Kiểm tra và loại bỏ giá trị khuyết thiếu
print(df.isnull().sum())
df = df.dropna(subset=['Comment'])

Star Rating    0
Comment        0
Sentiment      0
dtype: int64


In [5]:
# Kiểm tra và loại bỏ dữ liệu trùng lặp
duplicate_comments = df[df.duplicated(['Comment'])]
print("Các dòng dữ liệu trùng lặp trong cột 'Comment':")
print(duplicate_comments)
df = df.drop_duplicates(['Comment'])
print("Shape after dropping duplicates:", df.shape)

Các dòng dữ liệu trùng lặp trong cột 'Comment':
      Star Rating                                   Comment Sentiment
48              4                                   Rất tốt  tích cực
76              5                                    Rất ok  tích cực
107             2                             pin tụt nhanh  tiêu cực
135             5                              sản phẩm tốt  tích cực
173             2                             hao pin nhanh  tiêu cực
177             5                              sản phẩm tốt  tích cực
181             5                                    Rất ok  tích cực
183             5                              sản phẩm tốt  tích cực
192             5                               sản phẩm ok  tích cực
205             5                               máy dùng ok  tích cực
225             4                                       Tốt  tích cực
227             5                               sản phẩm ok  tích cực
237             5                         

In [6]:
# Chuẩn hóa và làm sạch văn bản
def remove_special_characters(text):
    # Loại bỏ các ký tự đặc biệt, giữ lại chữ cái, số, và các dấu câu
    return re.sub(r'[^a-zA-ZÀ-ỹà-ỹ0-9\s]', '', text)

def to_lowercase(text):
    # Chuyển đổi văn bản về chữ thường
    return text.lower()

def normalize_text(text):
    text = remove_special_characters(text)
    text = to_lowercase(text)
    return text

df['Comment'] = df['Comment'].apply(normalize_text)
print(df.head())

   Star Rating                                            Comment   Sentiment
0            4                      điện thoại này dùng rất thích    tích cực
1            4                               sử dụng thấy cũng ok    tích cực
2            2                       bảo hành ít quá chỉ 12 tháng    tiêu cực
3            5                              sản phẩm mượt chạy êm    tích cực
4            3  cho mình hỏi muốn khởi động lại máy hay tắt ng...  trung tính


In [12]:
from underthesea import word_tokenize

# Function to tokenize and build vocab using underthesea
def tokenize_and_build_vocab_vietnamese(comment):
    tokens = word_tokenize(comment, format="text")
    return tokens.split()

# Tokenize comments and build vocabulary
tokenized_comments = df['Comment'].apply(tokenize_and_build_vocab_vietnamese)
all_tokens = [token for sublist in tokenized_comments for token in sublist]
vocab = FreqDist(all_tokens)

# Build vocab_list from vocab
vocab_list = list(vocab.keys())

print("Tổng số từ duy nhất:", len(vocab_list))
print("Các từ và tần suất xuất hiện:")
for word, frequency in vocab.most_common(10):  # Print only top 10 words
    print(f"{word}: {frequency}")


Tổng số từ duy nhất: 2386
Các từ và tần suất xuất hiện:
máy: 715
không: 659
dùng: 593
mua: 472
được: 422
rất: 409
pin: 347
tốt: 338
mình: 322
thì: 319


In [15]:
import gensim
import numpy as np

# Assuming `model` is loaded using the new format for Gensim 4.0.0+
model = gensim.models.KeyedVectors.load_word2vec_format('vi_word2vec.txt', binary=False)

comments = df['Comment'].tolist()

# Xây dựng từ điển tra cứu từ trong bộ từ vựng
vocab_dict = {}
for word in vocab_list:
    vocab_dict[word] = True

# Mã hóa văn bản
encoded_comments = []
for comment in comments:
    encoded_sentence = []
    for word in comment.split():
        # Kiểm tra xem từ có trong bộ từ vựng và mô hình Word2Vec hay không
        if word in vocab_dict and word in model.key_to_index:  # Use key_to_index for checking existence
            # Lấy vector biểu diễn của từ từ mô hình
            encoded_sentence.append(model[word])
        else:
            # Sử dụng vector rỗng nếu không tìm thấy
            encoded_sentence.append(np.zeros(model.vector_size))
    encoded_comments.append(encoded_sentence)

# In ra ví dụ encoded_sentence để kiểm tra
print(encoded_comments[0]) 

[array([-0.08860478, -0.02230161, -0.0121149 , -0.00194529,  0.20760709,
       -0.09146854, -0.07412755,  0.03899725, -0.09914286, -0.15499985,
        0.2501074 , -0.05614415, -0.04446334, -0.04139058,  0.06852371,
       -0.27192336, -0.06798687,  0.1055525 ,  0.02714599,  0.00692284,
       -0.08111502, -0.09613015,  0.13130239,  0.05942395,  0.07362074,
       -0.00167992, -0.10843121, -0.05006169,  0.05630448,  0.03757608,
       -0.05354891,  0.07573725,  0.07735358,  0.04155717, -0.08398317,
       -0.04789606,  0.10648717,  0.04150081,  0.11151456,  0.04865672,
        0.07512528, -0.11729961, -0.00413176, -0.11899286,  0.00331507,
       -0.05433263, -0.05211951, -0.03734907, -0.13386998,  0.11181892,
       -0.04277122,  0.08043069,  0.02656305, -0.05665218,  0.13064016,
        0.11399975,  0.0347943 ,  0.02475453, -0.06676219, -0.0106067 ,
       -0.05376767, -0.13830021,  0.06179905,  0.06818796, -0.05136634,
       -0.18912354,  0.1023125 , -0.09921525, -0.04638093,  0.0